# Optimizing XGBoost parameters with all features of the prepared files

In [2]:
import pandas as pd
from xgboost import XGBClassifier
from sklift.models import ClassTransformation
from sklift import metrics
from sklift.viz import plot_qini_curve
import chime
from sklearn.model_selection import KFold
from statistics import mean, stdev
from sklearn.model_selection import GridSearchCV

/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
from tqdm.notebook import tqdm_notebook

In [4]:
%load_ext chime

In [1]:
best_features = ['first_redeem_date',
 'last_month_spend',
 'purchased_own_products',
 'gender_F',
 'express_points_spent_last_month',
 'regular_points_spent_sum',
 'bought_most_pop',
 'express_points_received_last_month',
 'gender_M',
 'no_redeem',
 'express_points_received_sum',
 'regular_points_received_sum',
 'regular_points_spent_mean',
 'days_since_last_transact',
 'relative_spend_last_month',
 'regular_points_spent_last_month',
 'age',
 'nunique',
 'express_points_spent_mean',
 'redeem_delay',
 'monthly_spend_mean']

In [5]:
df = pd.read_csv('train.csv', index_col='client_id')\
    .join(pd.read_csv('clients_train_modified_2.csv', index_col='client_id'),
          how='left')
df

,treatment_flg,purchased,regular_points_received_mean,regular_points_received_sum,express_points_received_mean,express_points_received_sum,regular_points_spent_mean,regular_points_spent_sum,express_points_spent_mean,express_points_spent_sum,...,last_month_spend,relative_spend_last_month,regular_points_received_last_month,express_points_received_last_month,regular_points_spent_last_month,express_points_spent_last_month,nunique,bought_most_pop,days_since_last_transact,purchased_own_products
client_id,,,,,,,,,,,,,,,,,,,,,
ad6561e2d8,1,1,2.432500,97.3,0.000000,0.0,-1.600000,-64.0,0.000000,0.0,...,2721.15,0.965714,1.966667,0.0,0.0,0.0,40,1.0,2,1.0
7c1ccbf93f,1,1,4.700000,9.4,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,851.00,0.900000,4.700000,0.0,0.0,0.0,2,1.0,13,1.0
b58fadcab6,1,1,2.772727,91.5,0.000000,0.0,-9.363636,-309.0,0.000000,0.0,...,2695.44,0.735432,3.050000,0.0,0.0,0.0,33,1.0,2,1.0
e99e6fabb9,0,0,1.046154,13.6,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,927.00,1.140460,1.100000,0.0,0.0,0.0,13,0.0,3,1.0
27fb6f8520,1,1,2.740000,82.2,0.666667,20.0,-0.133333,-4.0,-1.666667,-50.0,...,4200.60,1.487132,4.137500,0.0,0.0,0.0,30,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999d284453,1,1,8.625000,34.5,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,471.00,0.611551,2.300000,0.0,0.0,0.0,4,0.0,3,1.0
f634deea4e,0,1,1.086792,57.6,0.000000,0.0,-1.754717,-93.0,0.000000,0.0,...,3535.78,1.296013,1.088889,0.0,0.0,0.0,53,1.0,0,1.0
16cb4f99b0,0,1,3.485714,24.4,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,600.00,0.700637,1.450000,0.0,0.0,0.0,7,0.0,3,1.0


In [6]:
X = df[best_features]
X

,first_redeem_date,last_month_spend,purchased_own_products,gender_F,express_points_spent_last_month,regular_points_spent_sum,bought_most_pop,express_points_received_last_month,gender_M,no_redeem,...,regular_points_received_sum,regular_points_spent_mean,days_since_last_transact,relative_spend_last_month,regular_points_spent_last_month,age,nunique,express_points_spent_mean,redeem_delay,monthly_spend_mean
client_id,,,,,,,,,,,,,,,,,,,,,
ad6561e2d8,17674,2721.15,1.0,1,0.0,-64.0,1.0,0.0,0,0,...,97.3,-1.600000,2,0.965714,0.0,50.0,40,0.000000,171,2817.759130
7c1ccbf93f,17584,851.00,1.0,1,0.0,0.0,1.0,0.0,0,0,...,9.4,0.000000,13,0.900000,0.0,24.0,2,0.000000,104,945.555556
b58fadcab6,17789,2695.44,1.0,0,0.0,-309.0,1.0,0.0,0,0,...,91.5,-9.363636,2,0.735432,0.0,36.0,33,0.000000,317,3665.112931
e99e6fabb9,17681,927.00,1.0,1,0.0,0.0,0.0,0.0,0,0,...,13.6,0.000000,3,1.140460,0.0,79.0,13,0.000000,14,812.830189
27fb6f8520,17515,4200.60,1.0,1,0.0,-4.0,1.0,0.0,0,0,...,82.2,-0.133333,0,1.487132,0.0,34.0,30,-1.666667,151,2824.631429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999d284453,17992,471.00,1.0,0,0.0,0.0,0.0,0.0,0,0,...,34.5,0.000000,3,0.611551,0.0,72.0,4,0.000000,157,770.172414
f634deea4e,17602,3535.78,1.0,0,0.0,-93.0,1.0,0.0,0,0,...,57.6,-1.754717,0,1.296013,0.0,35.0,53,0.000000,36,2728.197414
16cb4f99b0,18047,600.00,1.0,1,0.0,0.0,0.0,0.0,0,0,...,24.4,0.000000,3,0.700637,0.0,50.0,7,0.000000,462,856.363636


In [7]:
treat = df.treatment_flg
treat

client_id
ad6561e2d8    1
7c1ccbf93f    1
b58fadcab6    1
e99e6fabb9    0
27fb6f8520    1
             ..
999d284453    1
f634deea4e    0
16cb4f99b0    0
23c2b72b2e    1
1430184499    0
Name: treatment_flg, Length: 140027, dtype: int64

In [8]:
y = df.purchased
y

client_id
ad6561e2d8    1
7c1ccbf93f    1
b58fadcab6    1
e99e6fabb9    0
27fb6f8520    1
             ..
999d284453    1
f634deea4e    1
16cb4f99b0    1
23c2b72b2e    1
1430184499    1
Name: purchased, Length: 140027, dtype: int64

In [9]:
uplift_scorer = metrics.make_uplift_scorer("qini_auc_score", treat)

# Model

## 1st optimization round

In [10]:
estimator = XGBClassifier(random_state=8, 
                          use_label_encoder=False)

In [11]:
model = ClassTransformation(estimator = estimator)

In [12]:
params = {
    'estimator__n_estimators': [10, 25, 50, 100],
    'estimator__learning_rate': [0.02, 0.05, 0.1],
    'estimator__max_depth': [1, 2, 3, 4],
    'estimator__reg_alpha': [0, 0.05],
    'estimator__reg_lambda': [0, 0.05]
}

In [13]:
grid_search = GridSearchCV(model,
                           param_grid=params,
                           scoring=uplift_scorer,
                           cv=5,
                           verbose=4
                          )

In [14]:
%%time
%%chime
grid_search = grid_search.fit(
    X=X,
    y=y,
    treatment=treat
)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[15:07:40] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 1/5] END estimator__learning_rate=0.02, estimator__max_depth=1, estimator__n_estimators=10, estimator__reg_alpha=0, estimator__reg_lambda=0;, score=0.006 total time=   0.4s
[15:07:41] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 2/5] END estimator__learning_rate=0.02, estimator__max_depth=1, estimator__n_estimators=10, estimator__reg_alpha=0, estimator__reg_lambda=0;, score=0.005 total time=   0.3s
[15:07:41] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 4/5] END estimator__learning_rate=0.1, estimator__max_depth=4, estimator__n_estimators=100, estimator__reg_alpha=0.05, estimator__reg_lambda=0.05;, score=0.024 total time=   4.6s
[15:31:49] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 5/5] END estimator__learning_rate=0.1, estimator__max_depth=4, estimator__n_estimators=100, estimator__reg_alpha=0.05, estimator__reg_lambda=0.05;, score=0.023 total time=   4.6s
[15:31:54] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


CPU times: user 1h 31min 32s, sys: 1min 29s, total: 1h 33min 2s
Wall time: 24min 14s


In [15]:
grid_search.best_score_

0.026604074826867174

In [16]:
best_params_1 = grid_search.best_params_
best_params_1

{'estimator__learning_rate': 0.05,
 'estimator__max_depth': 3,
 'estimator__n_estimators': 25,
 'estimator__reg_alpha': 0.05,
 'estimator__reg_lambda': 0}

# Best parameters are the same as before